<a href="https://colab.research.google.com/github/garthajon/QuantFinanceIntro/blob/main/zscore_stock_screener_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

# ------------------ Fetch S&P 500 tickers ------------------
sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
    )
}
resp = requests.get(sp500_url, headers=headers)
sp500_table = pd.read_html(resp.text)[0]

# Normalize tickers for yfinance (BRK.B -> BRK-B)
tickers = [s.replace(".", "-").strip() for s in sp500_table['Symbol']]

# ------------------ Fetch daily close price data ------------------
end_date = datetime.today()
start_date = end_date - timedelta(weeks=52*1)  # 1 years

print(f"Downloading {len(tickers)} tickers (1 years daily)...")
data = yf.download(tickers, start=start_date, end=end_date, interval="1d", progress=False)["Close"]

# Drop tickers with incomplete data
data = data.dropna(axis=1, how="any")
available_tickers = list(data.columns)
print(f"{len(available_tickers)} tickers with complete data.\n")

# ------------------ Compute weekly annualized volatility ------------------
returns = data.pct_change().dropna()
weekly_std = returns.resample('W').std() * np.sqrt(252)  # annualized volatility

# ------------------ Compute Z-score (most recent week) ------------------
zscore_params = {}
for ticker in available_tickers:
    mu = weekly_std[ticker].mean()
    sigma = weekly_std[ticker].std(ddof=0)
    zscore_params[ticker] = (mu, sigma)

latest_weekly_std = weekly_std.iloc[-1]

zscores = pd.Series({t: (latest_weekly_std[t] - zscore_params[t][0]) / zscore_params[t][1]
                     for t in available_tickers})

# ------------------ Screener: only extreme tickers ------------------
extreme_z = zscores[(zscores > 2) | (zscores < -2)].sort_values(ascending=False)

# ------------------ Display screener ------------------
print("\n=== S&P 500 STOCK SCREENER: Z > 2 OR Z < -2 ===\n")
print(extreme_z.to_frame("Z-Score"))


/tmp/ipython-input-168976174.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sp500_table = pd.read_html(resp.text)[0]
/tmp/ipython-input-168976174.py:26: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date, end=end_date, interval="1d", progress=False)["Close"]


503 tickers with complete data.


=== S&P 500 STOCK SCREENER: Z > 2 OR Z < -2 ===

       Z-Score
K     5.903309
CTVA  3.407544
FICO  3.304776
AES   3.303453
MRK   3.057374
PFE   2.921064
BIIB  2.704487
TMO   2.436415
NWS   2.313112
ABBV  2.212476
HUM   2.107414
